# Jpype库详解
　　参考：[python安装Jpype](https://blog.csdn.net/weixin_39598206/article/details/88665502)<br>
　　　　　[Jpype(PyPi地址)](https://pypi.org/project/JPype1/)<br>
　　　　　[Jpype官方地址](http://jpype.sourceforge.net/)<br>
　　　　　[Jpype版本库](https://pypi.org/project/JPype1/#files)<br>
　　　　　[JPype入门篇 -- Python调用Java代码并执行“Hello World”](https://blog.csdn.net/qq_25305833/article/details/124526875)<br>
　　　　　[]()<br>

## 一、简介
　　JPype是一个能够让 python 代码方便地调用 Java 代码的工具，从而克服了 python 在某些领域（如服务器端编程）中的不足。<br>
　　JPype的实际运行环境仍然是python runtime，只是在运行期间启动了一个嵌入的jvm。<br>

## 二、安装Jpype
**安装：**<br>
环境：Python3.6<br>
JAVA（Jar包）安装：<br>
　　1、需要下载好JDK，且版本要大于1.7，配置好环境变量JAVA_HOME。
　　2、如果JDK 是64 Bit，python解释器要64Bit （或者都是32 Bit）版本不一致会导致JVM运行时内存泄漏报异常：Process finished with exit code -1073741819 (0xC0000005)
VS C++：<br>
　　可在控制面板 > 程序内查看C++安装情。<br>
　　下载VS C++ 插件，http://landinghub.visualstudio.com/visual-cpp-build-tools 安装完成后，注册系统环境变量。<br>
　　根据提示，在已经安装的VS C++ tool的cmd prompt 中，输入 pip install jpype1 安装。<br>
注意：JAVA 与 Python 位数不一样，可能导致内存泄漏错误（Error: Process finished with exit code -1073741819(0xC0000005)）,建议都装64位。<br> 
>pip install jpype1　　或安装whl包：  pip install  \*\*.whl<br>

注意：将安装包放在Python包的路径下，或指定的包环境路径下。cmd操作到该文件路径下，或在文件路径框输入cmd 或者 在文件路径下快捷键：ctrl+shift 右键
调出shell窗口进行如下操作：<br>
>eg: pip install  JPype1_py3-0.5.5.2-cp35-none-win_amd64.whl<br>

　　若没有安装visual C++编译器， 则会报错（如Error：Microsoft Visual C++ 10.0 is required(Unable to find vcvarsall.bat)）。<br>
**测试安装成功：**<br>
　　下面是一个简单的 python 程序hello world ，通过 JPype 调用 Java 的打印函数，打印出字符串。<br>
>import jpype<br>
 　jvmPath = jpype.getDefaultJVMPath()<br>
 　jpype.startJVM(jvmPath)<br>
 　jpype.java.lang.System.out.println( “ hello world! ” )<br>
 　jpype.shutdownJVM()<br>

## 三、Jpype使用
### 3.1、启动 jvm
　　Jpype 提供的 startJVM(jvm, \*args) 函数的作用是启动java虚拟机，在代码中调用java代码之前，首先要使用此方法来启动java虚拟机。  
　　jvm ： 描述你系统中 jvm.dll 文件所在的路径，如“ C:\Program Files\IBM\Java50\jre\bin\j9vm\jvm.dll ”。可以通过调用 jpype.getDefaultJVMPath() 得到默认的 JVM 路径。  
　　args : 为可选参数，会被 JPype 直接传递给 JVM 作为 Java 虚拟机的启动参数。此处适合所有合法的 JVM 启动参数，例如：
-agentlib:libname[=options] -classpath classpath -verbose -Xint  

### 3.2、关闭 jvm
　　shutdownJVM() 函数的作用是关闭java虚拟机，在结束调用java代码时候，使用此方法可以关闭java虚拟机。

### 3.3、判断java虚拟机是否启动
　　JPype 提供的 jpype.isJVMStarted() 函数可以判断 JVM 是否已启动。返回值为 True 表示 JVM 已经启动，返回值为 False 表示 JVM 还未启动。　　
  
### 3.4、使用示例
　　hello world 程序
```python
import jpype 
jvmPath = jpype.getDefaultJVMPath() 
jpype.startJVM(jvmPath) 
jpype.java.lang.System.out.println( “ hello world! ” ) 
jpype.shutdownJVM()
```
### 3.5、Jpype调用jar包
　　jar包源代码如下，并打包编译为：JpypeDemo.jar。
```java
package wwz.bigeye;

public class JpypeDemo {
    public String sayHello(String user){
        return "hello" + user;
    }
    public int calc(int a, int b){
        return a + b;
    }
}
```
　　python调用代码
```python
# Djava.class.path指定jar包的路径
startJVM(getDefaultJVMPath(), "-ea", "-Djava.class.path=%s" % (os.getcwd() + '\JpypeDemo.jar'))

# 找到class
JYDClass = JClass("wwz.bigeye.JpypeDemo")
# 执行类中的函数
result = (JYDClass.calc(1, 2))
print(result)
shutdownJVM()
```


### 1、报错处理
#### 1、关于JPype报FileNotFoundError: [Errno 2] No such file or directory: '/usr/lib/jvm'错误的解决
　　找不到jvm，大概原因是环境变量没有生效。<br>
　　平时在命令行中能直接运行java，是因为java所在的bin目录被添加到PATH，且由export PATH后作为环境变量生效。但是JAVA_HOME只是作为普通变量，使用os.getenv()的时候获取环境变量时找不到JAVA_HOME，所以推测应该只要将JAVA_HOME前面添加export，重启项目即可。<br>
　　**解决办法：**<br>
　　　　在全局配置文件/etc/profile或个人配置文件~/.bashrc或~/.bash_profile中添加export JAVA_HOME即可，如下是我的/etc/profile的设置：<br>
># set java path <br>
export JAVA_HOME=/usr/local/java/latest<br>
export PATH=\\$\{JAVA_HOME\}/bin:\\${PATH}<br>
export CLASSPATH=.:\\$JAVA_HOME/lib/dt.jar:\\$JAVA_HOME/lib/tools.jar<br>
其他注意事项：如果使用PyCharm远程调试，若遇到同样报错，需要在导入pyhanlp前先设置环境变量，如下<br>
<br>
\# 设置环境变量<br>
import os<br>
os.environ\['JAVA_HOME'\] = '/usr/local/java/latest'<br>
<br>
\# 再导入JPype 即可避免无法找到java的问题<br>
import jpype<br>

In [1]:
import jpype


# 指定jar包位置, 或者.class文件
# class_path = 'E:/SinaifMock/utils/Test.class'
jar_path = "E:\create_TestAccount/test.jar"

# JVM的路径位置
JVM_path = "E:/java/jdk1.8.0_121/jre/bin/server/jvm.dll"

# 开启JVM，且指定jar包, 或者.class文件位置
jpype.startJVM(JVM_path, jar_path)

# 打印hello, word
jpype.java.lang.System.out.println("hello World")

# 引入java程序中的类.路径应该是项目中的package包路径.类名
javaClass = jpype.JClass('Test')

# 创建一个对象
javaInstance = javaClass()
# 执行类中的函数了
num = javaInstance.getSum(10, 20)
string = javaInstance.getString("getString")
print(num, string)

# 关闭JVM
jpype.shutdownJVM()

ModuleNotFoundError: No module named 'jpype'